In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import zscore
from sklearn.cluster import KMeans
import seaborn as sns

# Read the data
df = pd.read_csv('transaction_data.csv')

# Data preprocessing
def preprocess_data(df):
    # Convert date columns to datetime (DATE is in Excel format, need to convert from integer)
    df['DATE'] = pd.to_datetime(df['DATE'], unit='D', origin='1899-12-30')
    
    # Add derived time features
    df['MONTH'] = df['DATE'].dt.month
    df['DAY_OF_WEEK'] = df['DATE'].dt.dayofweek
    df['IS_WEEKEND'] = df['DAY_OF_WEEK'].isin([5, 6]).astype(int)
    
    # Calculate price per unit
    df['PRICE_PER_UNIT'] = df['TOT_SALES'] / df['PROD_QTY']
    
    # Flag premium purchases (above 75th percentile of price per unit)
    df['IS_PREMIUM'] = (df['PRICE_PER_UNIT'] > df['PRICE_PER_UNIT'].quantile(0.75)).astype(int)
    
    return df

def analyze_basket_affinity(df):
    """Analyze which products are commonly bought together"""
    basket_data = df.groupby(['TXN_ID', 'PROD_NAME'])['PROD_QTY'].sum().unstack().fillna(0)
    basket_data = (basket_data > 0).astype(int)
    
    product_pairs = []
    products = basket_data.columns
    
    for i in range(len(products)):
        for j in range(i+1, len(products)):
            prod1, prod2 = products[i], products[j]
            together = ((basket_data[prod1] == 1) & (basket_data[prod2] == 1)).sum()
            if together > 0:
                confidence = together / basket_data[prod1].sum()
                product_pairs.append({
                    'product1': prod1,
                    'product2': prod2,
                    'together_count': together,
                    'confidence': confidence
                })
    
    return pd.DataFrame(product_pairs).sort_values('confidence', ascending=False)

def analyze_customer_segments(df):
    """Advanced customer segmentation using KMeans clustering"""
    customer_features = df.groupby('LYLTY_CARD_NBR').agg({
        'TOT_SALES': ['sum', 'mean'],
        'PROD_QTY': ['sum', 'mean'],
        'IS_WEEKEND': 'mean',
        'IS_PREMIUM': 'mean',
        'TXN_ID': 'count'
    })
    
    # Normalize features
    features_normalized = customer_features.apply(zscore)
    
    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=4, random_state=42)
    customer_features['Segment'] = kmeans.fit_predict(features_normalized)
    
    return customer_features

def analyze_seasonal_patterns(df):
    """Analyze seasonal and temporal patterns"""
    seasonal = df.groupby(['MONTH', 'DAY_OF_WEEK']).agg({
        'TOT_SALES': 'sum',
        'PROD_QTY': 'sum',
        'TXN_ID': 'nunique'
    }).reset_index()
    
    # Add month names for better readability
    seasonal['MONTH_NAME'] = seasonal['MONTH'].map({
        1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
        7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
    })
    
    return seasonal

def analyze_product_performance(df):
    """Advanced product performance analysis"""
    product_metrics = df.groupby('PROD_NAME').agg({
        'TOT_SALES': 'sum',
        'PROD_QTY': 'sum',
        'LYLTY_CARD_NBR': 'nunique',
        'IS_PREMIUM': 'mean'
    }).reset_index()
    
    # Calculate additional metrics
    product_metrics['avg_price'] = product_metrics['TOT_SALES'] / product_metrics['PROD_QTY']
    product_metrics['customer_penetration'] = product_metrics['LYLTY_CARD_NBR'] / df['LYLTY_CARD_NBR'].nunique()
    
    return product_metrics

def print_advanced_insights(df, product_metrics, basket_affinity, customer_segments, seasonal):
    print("\n=== ADVANCED MARKETING INSIGHTS REPORT ===\n")
    
    print("1. PRODUCT PORTFOLIO INSIGHTS")
    print("--------------------------")
    top_products = product_metrics.nlargest(3, 'TOT_SALES')
    for _, prod in top_products.iterrows():
        print(f"- {prod['PROD_NAME']}")
        print(f"  Revenue: ${prod['TOT_SALES']:,.2f}")
        print(f"  Customer Penetration: {prod['customer_penetration']*100:.1f}%")
        print(f"  Premium Index: {prod['IS_PREMIUM']*100:.1f}%")
    
    print("\n2. CUSTOMER SEGMENT PROFILES")
    print("--------------------------")
    for segment in range(4):
        segment_data = customer_segments[customer_segments['Segment'] == segment]
        print(f"\nSegment {segment+1}:")
        print(f"- Size: {len(segment_data)} customers")
        print(f"- Avg Spend: ${segment_data['TOT_SALES']['sum'].mean():,.2f}")
        print(f"- Premium Purchase Rate: {segment_data['IS_PREMIUM']['mean'].mean()*100:.1f}%")
        print(f"- Weekend Shopping Rate: {segment_data['IS_WEEKEND']['mean'].mean()*100:.1f}%")
    
    print("\n3. PRODUCT AFFINITY INSIGHTS")
    print("--------------------------")
    top_pairs = basket_affinity.head(3)
    for _, pair in top_pairs.iterrows():
        print(f"- {pair['product1']} + {pair['product2']}")
        print(f"  Confidence: {pair['confidence']*100:.1f}%")
        print(f"  Joint Purchases: {pair['together_count']}")

def plot_advanced_insights(df, seasonal, customer_segments):
    plt.figure(figsize=(15, 10))
    
    # Plot 1: Seasonal Sales Pattern
    plt.subplot(221)
    seasonal_pivot = seasonal.pivot(index='MONTH_NAME', columns='DAY_OF_WEEK', values='TOT_SALES')
    sns.heatmap(seasonal_pivot, cmap='YlOrRd')
    plt.title('Sales Heatmap by Month and Day of Week')
    
    # Plot 2: Customer Segment Characteristics
    plt.subplot(222)
    segment_metrics = customer_segments.groupby('Segment').mean()
    segment_metrics[('TOT_SALES', 'mean')].plot(kind='bar')
    plt.title('Average Transaction Value by Segment')
    
    # Plot 3: Premium vs Regular Purchase Patterns
    plt.subplot(223)
    df.groupby('MONTH')['IS_PREMIUM'].mean().plot()
    plt.title('Premium Purchase Rate by Month')
    
    # Plot 4: Weekend vs Weekday Sales
    plt.subplot(224)
    df.groupby('IS_WEEKEND')['TOT_SALES'].mean().plot(kind='bar')
    plt.title('Average Sales: Weekday vs Weekend')
    
    plt.tight_layout()
    plt.show()

# Main analysis execution
df = preprocess_data(df)
product_metrics = analyze_product_performance(df)
basket_affinity = analyze_basket_affinity(df)
customer_segments = analyze_customer_segments(df)
seasonal = analyze_seasonal_patterns(df)

# Generate insights and visualizations
print_advanced_insights(df, product_metrics, basket_affinity, customer_segments, seasonal)
plot_advanced_insights(df, seasonal, customer_segments)